In [1]:
#使用到的套件
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
#讀 Var_Name sheet以求headers
headers = pd.read_excel ('MDS_Assignment2_Steelplates.xlsx', header=None, sheet_name = 'Var_Name')
headers = headers.transpose()
headers = list(headers.iloc[0])

#定義X,y
df = pd.read_excel ('MDS_Assignment2_Steelplates.xlsx', header=None, names=headers,sheet_name = 'Faults' )
X = df.iloc[ : , 0:26 ]
y = df['Bumps']

In [3]:
#用 X,  fit OLS
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Bumps   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     23.36
Date:                Tue, 10 Nov 2020   Prob (F-statistic):           1.75e-92
Time:                        16:20:29   Log-Likelihood:                -742.54
No. Observations:                1941   AIC:                             1537.
Df Residuals:                    1915   BIC:                             1682.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.99

In [4]:
#將小於0.01的 pvalue由小到大排序
results.pvalues.sort_values(ascending=True)
sigpar = results.pvalues.where(results.pvalues < 0.01).dropna()
print(sigpar)

const                 8.598540e-08
Y_Minimum             5.047454e-03
Y_Maximum             5.047950e-03
X_Perimeter           7.356327e-04
Y_Perimeter           4.356610e-04
Length_of_Conveyer    4.940961e-04
TypeOfSteel_A300      8.049098e-06
TypeOfSteel_A400      4.799347e-10
Edges_Index           2.126004e-07
Square_Index          2.292454e-14
Edges_Y_Index         2.385020e-05
Luminosity_Index      4.586082e-03
dtype: float64


In [14]:
#stepwise_forward regression
def forward_regression(X, y,threshold_in,verbose=False):
    
    #define included(list) 為選進的variable
    included = []
    while True:
        changed=False
        
        #define excluded(list)為included以外的variables
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded,dtype=np.dtype("float64"))
        
        #測試現有include內的variable和各個excluded內的variable一起跑OLS的狀況
        #選擇最小的pvalues作為best pvalue
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        
        #若best pvalue小於threshold_in，則將達成該best pvalue的excluded內的variable加入至include
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        else:
            break
    
    return included

In [15]:
#令X2為選取stepwise後挑選之variable的X資料
X2=X[forward_regression(X,y,0.01)]

Add  const                          with p-value 6.85051e-100
Add  TypeOfSteel_A300               with p-value 1.84163e-43
Add  Square_Index                   with p-value 1.19799e-23
Add  TypeOfSteel_A400               with p-value 1.5167e-22
Add  Edges_Index                    with p-value 1.98419e-16
Add  X_Minimum                      with p-value 3.18385e-10
Add  Y_Minimum                      with p-value 7.81275e-07
Add  Empty_Index                    with p-value 4.62997e-05
Add  Length_of_Conveyer             with p-value 0.00178672
Add  Edges_Y_Index                  with p-value 0.00826611
Add  Log_X_Index                    with p-value 0.000273934
Add  Sum_of_Luminosity              with p-value 0.00785379


In [17]:
#利用stepwise選出的重要變數跑OLS並印出adjusted R2
X2 = sm.add_constant(X2)
model = sm.OLS(y, X2)
results = model.fit()
print(results.rsquared_adj)

0.21415778289786813


In [18]:
#利用OLS之pvalue選出的重要變數跑OLS並印出adjusted R2
sigpar1=['const' ,'Y_Minimum','Y_Maximum','X_Perimeter' ,'Length_of_Conveyer',
 'TypeOfSteel_A300','TypeOfSteel_A400' ,'Edges_Index','Square_Index','Edges_Y_Index','Luminosity_Index']
X3=X[sigpar1]
X3 = sm.add_constant(X3)
model = sm.OLS(y, X3)
results = model.fit()
print(results.rsquared_adj)

0.20153804780511153
